In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    average_precision_score,
    roc_curve,
    precision_recall_curve
)

DATA_PATH = r"C:\MSAI\Machine Learning\Assignment 3\HIGGS.csv"
OUTPUT_DIR = r"C:\MSAI\Machine Learning\Assignment 3"
PERCENTAGES = [1, 2, 3, 4, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 100]

print("Data path:", DATA_PATH)


In [ ]:
def evaluate_classifier(clf, X_train, y_train, X_test, y_test):
    """
    Fits model on training data, predicts on test data,
    and returns metrics + training time.
    """
    import time
    start_time = time.time()
    clf.fit(X_train, y_train)
    train_time = time.time() - start_time

    y_pred = clf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1,
        "train_time": train_time
    }


In [ ]:
df = pd.read_csv(DATA_PATH, header=None)
print("Shape:", df.shape)

Y = df[0]
X = df.iloc[:, 1:]

X_train, X_test, y_train, y_test = train_test_split(
    X, Y,
    test_size=0.2,
    random_state=42,
    stratify=Y
)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Scaling complete!")


In [ ]:
results_lr = []
n_train = X_train_scaled.shape[0]

for p in PERCENTAGES:
    frac = p / 100.0
    n_samples = int(n_train * frac)

    X_sub = X_train_scaled[:n_samples]
    y_sub = y_train.iloc[:n_samples]

    print(f"[LR] Training with {p}% of training data ({n_samples} samples)")

    lr_clf = LogisticRegression(
        max_iter=1000,
        solver="saga",
        n_jobs=-1
    )

    metrics = evaluate_classifier(lr_clf, X_sub, y_sub, X_test_scaled, y_test)
    metrics["percentage"] = p
    metrics["n_samples"] = n_samples

    results_lr.append(metrics)

results_lr_df = pd.DataFrame(results_lr)
results_lr_df


In [ ]:
lr_report_path = os.path.join(OUTPUT_DIR, "LogisticRegressionReport.txt")
with open(lr_report_path, "w") as f:
    f.write(results_lr_df.to_string())

print("Logistic Regression report saved at:", lr_report_path)


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(results_lr_df["percentage"], results_lr_df["accuracy"], marker="o", label="Accuracy")
plt.plot(results_lr_df["percentage"], results_lr_df["precision"], marker="o", label="Precision")
plt.plot(results_lr_df["percentage"], results_lr_df["recall"], marker="o", label="Recall")
plt.plot(results_lr_df["percentage"], results_lr_df["f1"], marker="o", label="F1-score")
plt.xlabel("Training Data Used (%)")
plt.ylabel("Score")
plt.title("Logistic Regression Performance vs Sample Size (HIGGS)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 5))
plt.plot(results_lr_df["percentage"], results_lr_df["train_time"], marker="o")
plt.xlabel("Training Data Used (%)")
plt.ylabel("Training Time (seconds)")
plt.title("Logistic Regression Training Time vs Sample Size (HIGGS)")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
lr_full = LogisticRegression(
    max_iter=1000,
    solver="saga",
    n_jobs=-1,
    random_state=42
)
lr_full.fit(X_train_scaled, y_train)

y_score_full = lr_full.predict_proba(X_test_scaled)[:, 1]

# ROC
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_score_full)
roc_auc_full = roc_auc_score(y_test, y_score_full)

# PRC
prec_lr, rec_lr, _ = precision_recall_curve(y_test, y_score_full)
ap_full = average_precision_score(y_test, y_score_full)

print(f"LR ROC AUC: {roc_auc_full:.4f}")
print(f"LR AP (PR AUC): {ap_full:.4f}")
plt.figure(figsize=(8, 6))
plt.plot(fpr_lr, tpr_lr, label=f"LR (AUC = {roc_auc_full:.3f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve – Logistic Regression on HIGGS")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 6))
plt.plot(rec_lr, prec_lr, label=f"LR (AP = {ap_full:.3f})")
baseline = y_test.mean()
plt.hlines(baseline, 0, 1, colors='k', linestyles='--',
           label=f"Baseline (pos rate = {baseline:.2f})")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision–Recall Curve – Logistic Regression on HIGGS")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
